In [ ]:
import torch
from diffusers import Flux2Pipeline
from diffusers.utils import load_image
from PIL import Image

repo_id = "diffusers/FLUX.2-dev-bnb-4bit" #quantized text-encoder and DiT. VAE still in bf16
device = "cuda:0"
torch_dtype = torch.bfloat16

# ローカルでテキストエンコーダーを含む完全なパイプラインをロード
pipe = Flux2Pipeline.from_pretrained(
    repo_id, torch_dtype=torch_dtype
).to(device)

# メモリ最適化（オプション）
pipe.enable_model_cpu_offload()

prompt = "make the hand-drawn blueprint to a computer-created blueprint. Line drawing, with monochrome. delete all text, dimensions and unnecessary lines. make the line straight and clean. make the circle perfect. remove the shadow and background. only keep the blueprint lines."

# prompt = "convert the input image into a top-down view.."
# cat_image = load_image("https://huggingface.co/spaces/zerogpu-aoti/FLUX.1-Kontext-Dev-fp8-dynamic/resolve/main/cat.png")
input_image = Image.open('/home/localuser/Documents/flux2/l_ay4328_diymama03_fig04_w590.jpg')
image = pipe(
    prompt=prompt,  # prompt_embedsではなくpromptを直接渡す（ローカルでエンコード）
    image=[input_image], #optional multi-image input
    generator=torch.Generator(device=device).manual_seed(42),
    num_inference_steps=28, #28 steps can be a good trade-off
    guidance_scale=4
).images[0]

image.save("flux2_output_top_down.png")